In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
# Activate mypy type checking and black formatting
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter  # Activate import functionality for notebooks

# Chess-AI Statistik

Dieses Notebook lässt verschiedene AI-Versionen gegeneinander spielen und fasst die Ergebnisse in einer CSV-Datei zusammen.

Alle Optionen für die Ausführung können über die Umgebungsvariablen gesetzt werden. Das Notebook kann auch unbeaufsichtigt über [GitHub Actions](https://github.com/piuswalter/Chess-AI/actions/workflows/statistics.yml) ausgeführt werden. Der Workflow kann dort auf jedem existierenden Branch manuell gestartet werden. Der gewählte Branch entscheidet über den verwendeten Code. Ein Durchlauf mit den Standardoptionen (`development`-Branch) dauert derzeit etwa 1 Stunde. Die maximal erlaubte Laufzeit mit GitHub Actions beträgt 6 Stunden.

Verfügbare Versionen:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `simple evaluation function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `Exercise05AI`: Die fünfte Version der AI, welche die dritte Version um das Alpha-Beta-Pruning erweitert.
- `Exercise06AI`: Die sechste Version der AI, welche die fünfte Version erneut um Memoisierung erweitert.
- `Exercise07AI`: Die siebte Version der AI, welche die sechste Version um Progressive Deepening erweitert.
- `Exercise07LiteAI`: Eine Kopie der siebten Version, welche durch das Entfernen der Memoisierung deutlich weniger Speicher benötigt.
- `Exercise08AI`: Die achte Version der AI, welche die siebte Version um die Singular Value Extension erweitert.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish-Engine gesteuert wird.

## Optionen manuell anpassen

Die folgende Zelle kann dazu genutzt werden um die Optionen manuell anzupassen.

In [ ]:
import json

manual_config = json.loads(
    """
{
  "players": {
    "Exercise01AI": {},
    "Exercise02AI": {
      "search_depth": 3
    },
    "Exercise03AI": {
      "search_depth": 3
    },
    "Exercise04AI": {
      "search_depth": 3
    },
    "Exercise05AI": {
      "search_depth": 4
    },
    "Exercise06AI": {
      "search_depth": 4
    },
    "Exercise07AI": {
      "search_depth": 4
    },
    "Exercise07LiteAI": {
      "search_depth": 4
    },
    "Exercise08AI": {
      "search_depth": 5
    },
    "StockfishPlayer": {
      "elo": 1500,
      "time_limit": 0.1
    }
  },
  "opponents": {
    "Exercise01AI": {},
    "Exercise02AI": {
      "search_depth": 3
    },
    "Exercise03AI": {
      "search_depth": 3
    },
    "Exercise04AI": {
      "search_depth": 3
    },
    "Exercise05AI": {
      "search_depth": 4
    },
    "Exercise06AI": {
      "search_depth": 4
    },
    "Exercise07AI": {
      "search_depth": 4
    },
    "Exercise07LiteAI": {
      "search_depth": 4
    },
    "Exercise08AI": {
      "search_depth": 5
    },
    "StockfishPlayer": {
      "elo": 1500,
      "time_limit": 0.1
    }
  },
  "seed": 3,
  "repetitions": 3,
  "concurrent_games": 1,
  "exclude_reflexive_pairs": false,
  "make_symmetric_pairs": true
}
"""
)

In [ ]:
import os

# Import all versions
from AIBaseClass import ChessAI
from Exercise01AI import Exercise01AI
from Exercise02AI import Exercise02AI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI
from Exercise05AI import Exercise05AI
from Exercise06AI import Exercise06AI
from Exercise07AI import Exercise07AI
from Exercise07LiteAI import Exercise07LiteAI
from Exercise08AI import Exercise08AI
from StockfishPlayer import StockfishPlayer
from Main import run_games

available_versions = [
    Exercise01AI,
    Exercise02AI,
    Exercise03AI,
    Exercise04AI,
    Exercise05AI,
    Exercise06AI,
    Exercise07AI,
    Exercise07LiteAI,
    Exercise08AI,
    StockfishPlayer,
]

Die Methode `get_config` lädt eine gegebene Konfiguration aus den Umgebungsvariablen und gibt diese zurück.

In [ ]:
def get_config() -> dict:
    env_config = os.environ.get("config")
    if env_config:
        return json.loads(env_config)
    return manual_config

Die `cleanup_game_folder`-Funktion löscht alle Dateien im `games`-Ordner. Dies verhindert, dass bereits existierende Dateien ungewollt in den Ergebnissen enthalten sind.

In [ ]:
import os


def cleanup_game_folder() -> None:
    """Deletes all files inside the games folder."""
    for filename in os.listdir("../games"):
        if filename.endswith(".pgn") or filename.endswith(".csv"):
            os.remove(f"../games/{filename}")

Die Funktion `create_player_config` bekommt eine Spielerklasse, einen Spielernamen sowie die Konfiguration für alle Spieler und gibt die Spielerklasse zusammen mit ihrer Konfiguration zurück. Die Konfiguration ist dabei `immutable` damit sie in ein `set` hinzugefügt werden kann.

In [ ]:
def create_player_config(player: type, name: str, config: dict) -> tuple[type, tuple]:
    """Returns a player class with the contructor arguments as immutable objects."""
    return player, tuple({"player_name": name, **config[player.__name__]}.items())

Die Funktion `is_same_config` bekommt eine ChessAI-Klasse die sowohl Spieler, als auch Gegner ist und prüft ob die Konfiguration identisch ist. Diese Methode wird genutzt um doppelte Spielerpaare mit gleicher Konfiguration zu vermeiden.

In [ ]:
def is_same_config(player: type, config) -> bool:
    return config["players"][player.__name__] == config["opponents"][player.__name__]

Die Methode `build_pairs` bekommt zwei Listen von ChessAI-Klassen und eine Konfiguration als Dictionary. Sie bildet das kartesische Produkt der beiden Listen, wobei jeweils die Spieler-Konfiguration hinzugefügt wird.
Optional können dabei reflexive Paare (Spieler = Gegner) vermieden werden und symmetrische Paare (Farbenwechsel) erzeugt werden.

In [ ]:
def build_pairs(
    players: list[type], opponents: list[type], config: dict
) -> set[tuple[tuple[type, tuple], tuple[type, tuple]]]:
    """Builds cartesian pairs of two iterables"""
    make_symmetric_pairs = config["make_symmetric_pairs"]
    exclude_reflexive_pairs = config["exclude_reflexive_pairs"]
    result: set[tuple[ChessAI, ChessAI]] = set()
    for player in players:
        for opponent in opponents:
            if exclude_reflexive_pairs and player == opponent:
                continue
            player_1 = create_player_config(player, "White", config["players"])
            player_2 = create_player_config(opponent, "Black", config["opponents"])
            result.add((player_1, player_2))
            if make_symmetric_pairs and (
                not player in opponents or not is_same_config(player, config)
            ):
                s_player_1 = create_player_config(
                    opponent, "White", config["opponents"]
                )
                s_player_2 = create_player_config(player, "Black", config["players"])
                result.add((s_player_1, s_player_2))
    return result

Die `build_player_pairs`-Funktion ermittelt die angegebenen Spieler und Gegner, bildet eine Liste von Spielerpaaren und gibt diese nach dem Klassennamen sortiert zurück.


In [ ]:
def build_player_pairs(
    config: dict,
) -> list[tuple[tuple[type, tuple], tuple[type, tuple]]]:
    """Builds the sorted cartesian product of players x opponents and opponents x players"""
    # Get players and opponents
    players = [
        player for player in available_versions if player.__name__ in config["players"]
    ]
    opponents = [
        player
        for player in available_versions
        if player.__name__ in config["opponents"]
    ]
    # Build and return pairs
    player_pairs = build_pairs(players, opponents, config)
    return sorted(player_pairs, key=lambda x: (x[0][0].__name__, x[1][0].__name__))

Die `run`-Funktion nimmt ein Spielerpaar, eine Paarnummer, die Anzahl aller Paare sowie einen Seed und führt ein Spiel mit dem gewünschten Seed durch.

In [ ]:
def run(
    player_pair: tuple[tuple, tuple], pair_num: int, total_pairs: int, seed: int
) -> None:
    """Runs a game with the given player pair."""
    prefix = f"[{str(multiprocessing.current_process().pid)}]: "
    player_1, config_white = player_pair[0]
    player_2, config_black = player_pair[1]
    white_player = player_1(**dict(config_white))
    black_player = player_2(**dict(config_black))
    print(prefix + f"Starting game for player pair {pair_num + 1} of {total_pairs}")
    run_games(white_player, black_player, 1, seed)

Die `run_statistics`-Funktion nimmt eine Liste von Spielerpaaren und führt für jedes Paar die gewünschte Anzahl an Spielen mit dem gewünschten Seed durch. Falls angegeben werden die gewünschte Anzahl an Spielen parallel in mehreren Prozessen ausgeführt.

In [ ]:
from itertools import repeat
import multiprocessing


def run_statistics(player_pairs: list[tuple[ChessAI, ChessAI]], config: dict) -> None:
    """Runs one or more games with each player pair."""
    process_count = config["concurrent_games"]
    is_human_run = not bool(os.environ.get("CI", False))
    original_seed = config["seed"]
    repetitions = config["repetitions"]
    total_pairs = len(player_pairs)
    print(f"Total games: {total_pairs * repetitions}")
    for game_round in range(repetitions):
        seed = original_seed + game_round
        arguments = [
            (pair, i, total_pairs, seed) for i, pair in enumerate(player_pairs)
        ]
        print(
            f"Starting game round {game_round + 1} of {repetitions} using seed {seed}"
        )
        # Let all player pairs play
        if is_human_run:
            for args in arguments:
                run(*args)
        else:
            with multiprocessing.Pool(process_count) as executor:
                list(executor.starmap(run, arguments))

Die `generate_stats`-Funktion lädt die, als Kommentar gespeicherten, Statistiken für jeden Zug und bereitet sie für den CSV-Export auf.

In [ ]:
import chess.pgn


def generate_stats(nodes: chess.pgn.Mainline[chess.pgn.ChildNode]) -> dict:
    """Collects and generates stats from .pgn file comments"""
    stats = {}
    raw_stats: list[dict] = [json.loads("{" + node.comment + "}") for node in nodes]
    player_indices = lambda i: range(i, len(raw_stats), 2)
    stats_count_white = len(tuple(player_indices(0)))
    stats_count_black = len(tuple(player_indices(1)))
    stats["last_state"] = raw_stats[-2]["state"].split(".")[1]
    stats["engine_time_white"] = sum(raw_stats[i]["time"] for i in player_indices(0))
    stats["engine_time_black"] = sum(raw_stats[i]["time"] for i in player_indices(1))
    stats["move_time_white"] = stats["engine_time_white"] / stats_count_white
    stats["move_time_black"] = stats["engine_time_black"] / stats_count_black

    depth_list_white = [
        raw_stats[i]["eval_depth"]
        for i in player_indices(0)
        if "eval_depth" in raw_stats[i]
    ]
    stats["depth_average_white"] = sum(depth_list_white) / (len(depth_list_white) or 1)
    depth_list_black = [
        raw_stats[i]["eval_depth"]
        for i in player_indices(1)
        if "eval_depth" in raw_stats[i]
    ]
    stats["depth_average_black"] = sum(depth_list_black) / (len(depth_list_black) or 1)

    stats["cache_tries_white"] = sum(
        raw_stats[i].get("cache_tries", 0) for i in player_indices(0)
    )
    stats["cache_tries_black"] = sum(
        raw_stats[i].get("cache_tries", 0) for i in player_indices(1)
    )
    stats["cache_hits_white"] = sum(
        raw_stats[i].get("cache_hits", 0) for i in player_indices(0)
    )
    stats["cache_hits_black"] = sum(
        raw_stats[i].get("cache_hits", 0) for i in player_indices(1)
    )
    stats["move_list"] = ",".join(entry["move"] for entry in raw_stats)
    return stats

Die `collect_results`-Funktion importiert die Ergebnisse aus allen `.pgn`-Dateien im `games`-Ordner und bereitet sie als Dictionary auf.

In [ ]:
# Collect results from .pgn files


def collect_results() -> dict[str, list]:
    """Scans all .pgn files inside the games folder and collects the game results as dictionary."""
    # All results are presented from the view of the white color
    results_map_white = {"1-0": "Won", "0-1": "Lost", "1/2-1/2": "Remis", "*": "Remis"}
    results_map_black = {"0-1": "Won", "1-0": "Lost", "1/2-1/2": "Remis", "*": "Remis"}
    games_map: dict[str, list] = {}

    # Iterate over all .pgn files
    for filename in os.listdir("../games"):
        if not filename.endswith(".pgn"):
            continue
        with open(os.path.join("..", "games", filename)) as pgn:
            pgn_game = chess.pgn.read_game(pgn)
        if not pgn_game:
            continue

        # Collect results
        results = {
            "round": pgn_game.headers["Round"],
            "opponent": pgn_game.headers["Black"],
            "result_white": results_map_white[pgn_game.headers["Result"]],
            "result_black": results_map_black[pgn_game.headers["Result"]],
            "outcome": pgn_game.headers["Outcome"],
            "duration": pgn_game.headers["Duration"],
            "seed": pgn_game.headers["Seed"],
            "commit": pgn_game.headers["Commit"],
            "depth_white": pgn_game.headers["Depth"].split("-")[0],
            "depth_black": pgn_game.headers["Depth"].split("-")[1],
            "total_moves": pgn_game.end().ply(),
            "filename": filename,
            **generate_stats(pgn_game.mainline()),
        }

        # Save results (uses white player name as key)
        entry = games_map.get(pgn_game.headers["White"], [])
        entry.append(results)
        games_map[pgn_game.headers["White"]] = entry
    return games_map

Die `export_to_csv`-Funktion exportiert eine gegebene Sammlung (Dictionary) von Spielergebnissen als CSV-Datei.

In [ ]:
import csv
from datetime import datetime


def export_to_csv(games_map: dict[str, list]) -> None:
    """Exports the given results dictionary to csv."""
    filename = f'statistics_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")}.csv'
    with open(os.path.join("..", "games", filename), "w", newline="") as csv_file:
        csv_writer = csv.writer(
            csv_file, delimiter=";", quotechar='"', quoting=csv.QUOTE_MINIMAL
        )
        # Write headlines
        csv_writer.writerow(
            [
                "Player",
                "Result white",
                "Depth white",
                "Depth average white",
                "Total engine time white",
                "Average move time white",
                "Cache tries white",
                "Cache hits white",
                "Opponent",
                "Result black",
                "Depth black",
                "Depth average black",
                "Total engine time black",
                "Average move time black",
                "Cache tries black",
                "Cache hits black",
                "Outcome",
                "Moves",
                "Duration",
                "Last state",
                "Round",
                "Seed",
                "Commit Hash",
                "File",
                "Move list",
            ]
        )
        # Write results
        for player in sorted(games_map):
            games = sorted(games_map[player], key=lambda game: game["filename"])
            for game in games:
                csv_writer.writerow(
                    [
                        player,
                        game["result_white"],
                        game["depth_white"],
                        str(game["depth_average_white"]).replace(".", ","),
                        str(game["engine_time_white"]).replace(".", ","),
                        str(game["move_time_white"]).replace(".", ","),
                        game["cache_tries_white"],
                        game["cache_hits_white"],
                        game["opponent"],
                        game["result_black"],
                        game["depth_black"],
                        str(game["depth_average_black"]).replace(".", ","),
                        str(game["engine_time_black"]).replace(".", ","),
                        str(game["move_time_black"]).replace(".", ","),
                        game["cache_tries_black"],
                        game["cache_hits_black"],
                        game["outcome"].split(".")[1],
                        game["total_moves"],
                        str(game["duration"]).replace(".", ","),
                        game["last_state"],
                        game["round"].replace("/", " of "),
                        game["seed"],
                        game["commit"],
                        game["filename"],
                        game["move_list"],
                    ]
                )

Die folgende Zelle bereinigt den `games`-Ordner, erstellt die Spielerpaare, startet die Spiele, sammelt die Ergebnisse und exportiert sie als CSV-Datei.

In [ ]:
# Run and export games
if __name__ == "__main__":
    # cleanup_game_folder()
    config = get_config()
    player_pairs = build_player_pairs(config)
    run_statistics(player_pairs, config)
    results = collect_results()
    export_to_csv(results)